# Notebook Jupyter adapté pour le nettoyage et la transformation des jeu de données 

In [9]:
import json
import csv
import pandas as pd

### - Jeu de données ghostbikes (accident mortel)

##### 1. Transformation

In [8]:
# Fonction pour charger un fichier .geojson
def load_geojson(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# Fonction pour nettoyer les données
def clean_data(data):
    cleaned_data = []
    for feature in data.get("features", []):
        properties = feature.get("properties", {})
        geometry = feature.get("geometry", {})
        coordinates = geometry.get("coordinates", [])
        if len(coordinates) == 2:
            cleaned_data.append({
                "id": properties.get("@id"),
                "lat": coordinates[1],
                "lon": coordinates[0],
                "start_date": properties.get("start_date")
            })
    return cleaned_data

# Fonction pour écrire les données nettoyées dans un fichier CSV
def write_to_csv(cleaned_data, output_file):
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["id", "lat", "lon", "start_date"])
        writer.writeheader()
        writer.writerows(cleaned_data)

# Chemin d'entrée et de sortie
input_file = "export.geojson"
output_file = "ghostbikes.csv"

# Chargement, nettoyage et exportation
data = load_geojson(input_file)
cleaned_data = clean_data(data)
write_to_csv(cleaned_data, output_file)

##### 2. Finalisation

In [11]:
data_ghostbike = pd.read_csv("ghostbikes.csv", sep=",")
filtered_filtered_filtered_df_ghostbike = pd.DataFrame(data_ghostbike)
# Suppression de "node/" dans la colonne 'id'
df_ghostbike["id"] = df_ghostbike["id"].str.replace("node/", "")
# Ajout de la colonne 'statut_accident' avec la valeur "Mortel"
df_ghostbike["statut_accident"] = "Mortel"
print(df_ghostbike)

            id        lat       lon  start_date statut_accident
0   9611135117  45.180153  5.716451  2022-03-26          Mortel
1  11002892805  45.177253  5.718046  2023-06-24          Mortel


### - Jeu de données accident de vélo

##### Filtrage pour la commune de Grenoble (Code INSEE : 38185)

In [ ]:
data_accident_velo = pd.read_csv("accidentsVelo.csv", sep=",")
df_accident_velo = pd.DataFrame(data_accident_velo)
df_accident_velo.info()

# Filtrer les données selon le code INSEE de la commune de grenoble (38185)
filtered_df = df_accident_velo[(df_accident_velo["com"] == "38185")]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74758 entries, 0 to 74757
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Num_Acc          74758 non-null  int64  
 1   date             74758 non-null  object 
 2   an               74758 non-null  int64  
 3   mois             74758 non-null  object 
 4   jour             74758 non-null  object 
 5   hrmn             74758 non-null  object 
 6   dep              74758 non-null  object 
 7   com              74758 non-null  object 
 8   lat              74758 non-null  object 
 9   long             74490 non-null  object 
 10  agg              74758 non-null  int64  
 11  int              74758 non-null  int64  
 12  col              74754 non-null  float64
 13  lum              74758 non-null  int64  
 14  atm              74755 non-null  float64
 15  catr             74758 non-null  int64  
 16  circ             74615 non-null  float64
 17  nbv         

C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\3616175776.py:1: DtypeWarning: Columns (8,9,20,21,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_accident_velo = pd.read_csv("accidentsVelo.csv", sep=",")


(337, 39)

##### Standardisation

In [30]:
filtered_df["lat"] = filtered_df["lat"].astype(str).str.replace(",", ".")
filtered_df["long"] = filtered_df["long"].astype(str).str.replace(",", ".")

# Convertir les colonnes en type float pour une manipulation correcte (si nécessaire)
filtered_df["lat"] = filtered_df["lat"].astype(float)
filtered_df["long"] = filtered_df["long"].astype(float)

C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\3496815627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["lat"] = filtered_df["lat"].astype(str).str.replace(",", ".")
C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\3496815627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["long"] = filtered_df["long"].astype(str).str.replace(",", ".")
C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\3496815627.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

##### Création de colonnes pour mesurer la qualité des données

###### Analyse des heures

In [29]:
# Fonction pour vérifier si l'heure est cohérente
def verifier_heure(heure):
    try:
        # Séparer l'heure et les minutes
        h, m = map(int, heure.split(':'))
        # Vérifier si l'heure et les minutes sont dans les plages valides
        return "Cohérente" if 0 <= h <= 23 and 0 <= m <= 59 else "Non cohérente"
    except:
        return "Non cohérente"

# Appliquer la fonction sur la colonne hrmn pour créer une nouvelle colonne statut_heure
filtered_df["statut_heure"] = filtered_df["hrmn"].astype(str).apply(verifier_heure)
print(filtered_df)

            Num_Acc        date    an      mois      jour   hrmn dep    com  \
136    200500003397  2005-01-13  2005   janvier     jeudi   73:0  38  38185   
137    200500003400  2005-01-14  2005   janvier  vendredi  17:45  38  38185   
141    200500003415  2005-01-25  2005   janvier     mardi   90:5  38  38185   
339    200500007846  2005-02-18  2005   février  vendredi  17:30  38  38185   
637    200500015267  2005-03-30  2005      mars  mercredi  17:35  38  38185   
...             ...         ...   ...       ...       ...    ...  ..    ...   
60668  201700027686  2017-11-30  2017  novembre     jeudi  19:30  38  38185   
60669  201700027686  2017-11-30  2017  novembre     jeudi  19:30  38  38185   
64862  201800025689  2018-01-24  2018   janvier  mercredi  14:20  38  38185   
66393  201900008440  2019-04-23  2019     avril     mardi  12:30  38  38185   
67403  201900029141  2019-05-05  2019       mai  dimanche  17:10  38  38185   

              lat       long  ...  equipement  obs 

C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\281969821.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["statut_heure"] = filtered_df["hrmn"].astype(str).apply(verifier_heure)


###### Analyse des coordonnées

In [33]:
# Fonction pour vérifier la cohérence des coordonnées
def verifier_coord(lat, lon):
    if lat == 0 or lon == 0:
        return "Non cohérentes"
    if lat < 45 or lat > 46:
        return "Non cohérentes"
    if lon < 5 or lon > 6:
        return "Non cohérentes"
    return "Cohérentes"

# Appliquer la fonction ligne par ligne pour créer une nouvelle colonne
filtered_df["statut_coord"] = filtered_df.apply(lambda row: verifier_coord(row["lat"], row["long"]), axis=1)
print(filtered_df)

            Num_Acc        date    an      mois      jour   hrmn dep    com  \
136    200500003397  2005-01-13  2005   janvier     jeudi   73:0  38  38185   
137    200500003400  2005-01-14  2005   janvier  vendredi  17:45  38  38185   
141    200500003415  2005-01-25  2005   janvier     mardi   90:5  38  38185   
339    200500007846  2005-02-18  2005   février  vendredi  17:30  38  38185   
637    200500015267  2005-03-30  2005      mars  mercredi  17:35  38  38185   
...             ...         ...   ...       ...       ...    ...  ..    ...   
60668  201700027686  2017-11-30  2017  novembre     jeudi  19:30  38  38185   
60669  201700027686  2017-11-30  2017  novembre     jeudi  19:30  38  38185   
64862  201800025689  2018-01-24  2018   janvier  mercredi  14:20  38  38185   
66393  201900008440  2019-04-23  2019     avril     mardi  12:30  38  38185   
67403  201900029141  2019-05-05  2019       mai  dimanche  17:10  38  38185   

            lat     long  ...  obs  obsm  choc  man

C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\4172450745.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["statut_coord"] = filtered_df.apply(lambda row: verifier_coord(row["lat"], row["long"]), axis=1)


##### Utilisation des métadonnées pour remplacer les codes numériques par leur significations

In [35]:
# Pour agg
filtered_df["agg"] = filtered_df["agg"].replace({
    1: "Hors agglomération",
    2: "En agglomération"
})

# Pour int
filtered_df["int"] = filtered_df["int"].replace({
    0: "Non renseigné",
    1: "Hors intersection",
    2: "Intersection en X",
    3: "Intersection en T",
    4: "Intersection en Y",
    5: "Intersection à plus de 4 branches",
    6: "Giratoire",
    7: "Place",
    8: "Passage à niveau",
    9: "Autre intersection"
})

# Pour lum
filtered_df["lum"] = filtered_df["lum"].replace({
    1: "Plein jour",
    2: "Crépuscule ou aube",
    3: "Nuit sans éclairage public",
    4: "Nuit avec éclairage public non allumé",
    5: "Nuit avec éclairage public allumé"
})

# Pour atm
filtered_df["atm"] = filtered_df["atm"].replace({
    1: "Normale",
    2: "Pluie légère",
    3: "Pluie forte",
    4: "Neige - grêle",
    5: "Brouillard - fumée",
    6: "Vent fort - tempête",
    7: "Temps éblouissant",
    8: "Temps couvert",
    9: "Autre",
    -1: "Non renseigné"
})

# Pour circ
filtered_df["circ"] = filtered_df["circ"].replace({
    1: "A sens unique",
    2: "Bidirectionnelle",
    3: "A chaussées séparées",
    4: "Avec voies d'affectation variable",
    -1: "Non renseigné"
})

# Pour prof
filtered_df["prof"] = filtered_df["prof"].replace({
    1: "Plat",
    2: "Pente",
    3: "Sommet de côte",
    4: "Bas de côte",
    -1: "Non renseigné"
})

# Pour surf
filtered_df["surf"] = filtered_df["surf"].replace({
    1: "Normale",
    2: "Mouillée",
    3: "Flaques",
    4: "Inondée",
    5: "Enneigée",
    6: "Boue",
    7: "Verglacée",
    8: "Corps gras - huile",
    9: "Autre",
    -1: "Non renseigné"
})

# Pour situ
filtered_df["situ"] = filtered_df["situ"].replace({
    0: "Aucun",
    1: "Sur chaussée",
    2: "Sur bande d'arrêt d'urgence",
    3: "Sur accotement",
    4: "Sur trottoir",
    5: "Sur piste cyclable",
    6: "Sur autre voie spéciale",
    8: "Autres",
    -1: "Non renseigné"
})

# Pour grav
filtered_df["grav"] = filtered_df["grav"].replace({
    1: "Indemne",
    2: "Tué",
    3: "Blessé hospitalisé",
    4: "Blessé léger"
})

# Pour sexe
filtered_df["sexe"] = filtered_df["sexe"].replace({
    1: "Masculin",
    2: "Féminin"
})

# Pour trajet
filtered_df["trajet"] = filtered_df["trajet"].replace({
    0: "Non renseigné",
    1: "Domicile - travail",
    2: "Domicile - école",
    3: "Courses - achats",
    4: "Utilisation professionnelle",
    5: "Promenade - loisirs",
    9: "Autre",
    -1: "Non renseigné"
})

C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\2167096452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["agg"] = filtered_df["agg"].replace({
C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\2167096452.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["int"] = filtered_df["int"].replace({
C:\Users\alexm\AppData\Local\Temp\ipykernel_7060\2167096452.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [37]:
print(filtered_df["trajet"])

136                    Autre
137                    Autre
141            Non renseigné
339      Promenade - loisirs
637      Promenade - loisirs
                ...         
60668     Domicile - travail
60669    Promenade - loisirs
64862     Domicile - travail
66393          Non renseigné
67403    Promenade - loisirs
Name: trajet, Length: 337, dtype: object
